# 2 - Augmentation des données : Kilométrage
- **Source du jeu de données :** [Portail Open Data SNCF](https://ressources.data.sncf.com/explore/dataset/tarifs-tgv-inoui-ouigo/information/)
- **Producteur du jeu de données :** SNCF Voyageurs
- **Date de l'export :** Mercredi 18 septembre 2024

## 0 - Import des modules et des données

In [64]:
import pandas as pd

In [65]:
df_base = pd.read_csv("datasets/tarifs-tgv-inoui-ouigo.csv", sep=";")
df_km = pd.read_csv("datasets/emission-co2-tgv.csv", sep=";")

In [84]:
df_base.head()

,Gare origine,Gare origine - code UIC,Destination,Gare destination - code UIC,concat_uic
0,AEROPORT CDG2 TGV ROISSY,87271494,MARSEILLE ST CHARLES,87751008,8775100887271494
1,AEROPORT CDG2 TGV ROISSY,87271494,MONTPELLIER SUD DE FRANCE,87688887,8768888787271494
2,AIX EN PROVENCE TGV,87319012,LYON-SAINT EXUPERY TGV,87762906,8776290687319012
3,MARNE LA VALLEE CHESSY,87111849,AIX EN PROVENCE TGV,87319012,8731901287111849
4,MARNE LA VALLEE CHESSY,87111849,POITIERS,87575001,8757500187111849


In [66]:
df_km.head()

,Transporteur,Origine,Origine_uic,Destination,Destination_uic,Distance entre les gares,Train - Empreinte carbone (kgCO2e),Autocar longue distance - Empreinte carbone (kgCO2e),Avion - Empreinte carbone (kgCO2e),"Voiture électrique (2,2 pers.) - Empreinte carbone (kgCO2e)","Voiture thermique (2,2 pers.) - Empreinte carbone (kgCO2e)"
0,TGV,Paris Montparnasse,87391003,Rennes,87471003,364,1.3,10.5,79.8,16.5,37.8
1,TGV,Paris Montparnasse,87391003,Angers-St-Laud,87484006,303,1.1,9.0,68.4,14.1,32.5
2,TGV,Paris Montparnasse,87391003,Saint-Pierre-des-Corps,87571240,221,0.8,7.2,52.5,11.2,25.8
3,TGV,Paris Est,87113001,Strasbourg,87212027,451,1.6,14.6,104.0,22.9,52.7
4,TGV,Paris Est,87113001,Metz,87192039,352,1.2,10.0,73.6,15.6,35.9


In [67]:
df_km.columns

Index(['Transporteur ', 'Origine', 'Origine_uic', 'Destination ',
       'Destination_uic ', 'Distance entre les gares',
       'Train - Empreinte carbone (kgCO2e) ',
       'Autocar longue distance - Empreinte carbone (kgCO2e)',
       'Avion - Empreinte carbone (kgCO2e)',
       'Voiture électrique (2,2 pers.) - Empreinte carbone (kgCO2e)',
       'Voiture thermique (2,2 pers.) - Empreinte carbone (kgCO2e)'],
      dtype='object')

In [68]:
df_km = df_km[['Transporteur ', 'Origine', 'Origine_uic', 'Destination ',
       'Destination_uic ', 'Distance entre les gares',]]

In [69]:
df_km['Origine_uic'] = df_km['Origine_uic'].astype("str")
df_km['Destination_uic '] = df_km['Destination_uic '].astype("str")

In [70]:
df_km["concat_uic"] = df_km['Origine_uic']+ df_km['Destination_uic ']

In [73]:
df_km.head()

,Transporteur,Origine,Origine_uic,Destination,Destination_uic,Distance entre les gares,concat_uic
0,TGV,Paris Montparnasse,87391003,Rennes,87471003,364,8739100387471003
1,TGV,Paris Montparnasse,87391003,Angers-St-Laud,87484006,303,8739100387484006
2,TGV,Paris Montparnasse,87391003,Saint-Pierre-des-Corps,87571240,221,8739100387571240
3,TGV,Paris Est,87113001,Strasbourg,87212027,451,8711300187212027
4,TGV,Paris Est,87113001,Metz,87192039,352,8711300187192039


In [74]:
df_km["concat_uic"].nunique()

118

In [75]:
df_base.columns

Index(['Transporteur', 'Gare origine', 'Gare origine - code UIC',
       'Destination', 'Gare destination - code UIC', 'Classe',
       'Profil tarifaire', 'Prix minimum', 'Prix maximum'],
      dtype='object')

In [77]:
df_base = df_base[['Gare origine', 'Gare origine - code UIC', 'Destination', 'Gare destination - code UIC',]]

In [78]:
df_base['Gare destination - code UIC'] = df_base['Gare destination - code UIC'].astype("str")
df_base['Gare origine - code UIC'] = df_base['Gare origine - code UIC'].astype("str")

In [79]:
df_base["concat_uic"] = df_base['Gare destination - code UIC'] + df_base['Gare origine - code UIC']

In [81]:
df_base["concat_uic"].nunique()

4211

In [89]:
df_km[(df_km["Origine_uic"] == "87391003") & (df_km["Destination_uic "] == "87481002")]

,Transporteur,Origine,Origine_uic,Destination,Destination_uic,Distance entre les gares,concat_uic
26,TGV,Paris Montparnasse,87391003,Nantes,87481002,386,8739100387481002


In [93]:
df_km[(df_km["Origine_uic"] == "87481002") & (df_km["Destination_uic "] == "87391003")]

,Transporteur,Origine,Origine_uic,Destination,Destination_uic,Distance entre les gares,concat_uic


In [90]:
df_base[(df_base["Gare origine - code UIC"] == "87391003") & (df_base["Gare destination - code UIC"] == "87481002")]

,Gare origine,Gare origine - code UIC,Destination,Gare destination - code UIC,concat_uic
152,PARIS MONTPARNASSE 1 ET 2,87391003,NANTES,87481002,8748100287391003


In [94]:
df_base[(df_base["Gare origine - code UIC"] == "87481002") & (df_base["Gare destination - code UIC"] == "87391003")]

,Gare origine,Gare origine - code UIC,Destination,Gare destination - code UIC,concat_uic
3052,NANTES,87481002,PARIS MONTPARNASSE 1 ET 2,87391003,8739100387481002
3588,NANTES,87481002,PARIS MONTPARNASSE 1 ET 2,87391003,8739100387481002
7265,NANTES,87481002,PARIS MONTPARNASSE 1 ET 2,87391003,8739100387481002
8486,NANTES,87481002,PARIS MONTPARNASSE 1 ET 2,87391003,8739100387481002
10944,NANTES,87481002,PARIS MONTPARNASSE 1 ET 2,87391003,8739100387481002
12796,NANTES,87481002,PARIS MONTPARNASSE 1 ET 2,87391003,8739100387481002


In [92]:
df_base["concat_uic"].nunique()

4211